# Analysis and Visualization of Complex Agro-Environmental Data
---
## Regression

### 1. Run simple linear regression

Simple linear regression is the simplest form of regression analysis. It is also commonly used in Exploratory Data Analysis when we are interested in exploring if a given continuous response variable is affected by other independent (or predictor) variables. When we aim at modelling a response continuous variable with multiple linear regression using a big set of potential candidate variables, simple regression analysis is often used as a first filter to select a subset of candidate variables. Again, significant effect of a predictor on the response variable does not imply causation; but causation implies a significant an effect of a predictor on the response. Along with correlation analysis, regression is also important as a basis to establish hypothesis to be tested with more elaborated confirmatory statistics.

Linear regression models may be run with several python modules such as SciPy, statsmodel and scikit-learn.

##### Example with the [penguin](https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data) dataset

In [ ]:
# import the packages we are going to be using
import numpy as np # for getting our distribution
import pandas as pd # to handle data frames
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for plotting
from scipy import stats # to compute statistics

# import data ('penguin' dataset)
data = pd.read_csv('penguins_lter.csv')
data.drop(data.iloc[:,14:18], axis=1, inplace=True)
data = data.dropna()
data.info()

Let's regress the variable 'body mass' against 'culmen depth'

In [ ]:
# Use the `penguin` dataset to relate body mass as function of culmen depth (i.e. to infer body mass from culmen depth)

x=data["Culmen Depth (mm)"]
y=data["Body Mass (g)"]

# Execute a method that returns some important key values of Linear Regression:
slope, intercept, r, p, std_err = stats.linregress(x, y)

# plot data with fitted line
def myfunc(x):
  return intercept + slope * x # function that returns fitted values

mymodel = list(map(myfunc, x)) # apply function to each x value

plt.scatter(x, y)
plt.plot(x, mymodel, color="red")
plt.show()

The overal relationship tends to be negative. Now we will test the null hypothesis that the slope of relationship is zero

In [ ]:
# Run the regression test
print('slope estimate=%.2f, r-square=%.3f, p=%.6f' % (slope, r**2, p))
alpha=0.05
if p <= alpha:
 print('reject H0 that the slope of the relationship is = 0')
else:
 print('fail to reject H0 that the slope of the relationship is = 0')

The **statsmodel** module provides a more complete output using much less code:

In [ ]:
import statsmodels.api as sm

x2 = x
x2 = sm.add_constant(x2) # adding a constant (Intercept)

model = sm.OLS(y, x2).fit()
predictions = model.predict(x2) 

print_model = model.summary()
print(print_model)

# Some notes on model outputs:

# Omnibus tests are a kind of statistical test. They test whether the explained variance in a 
# set of data is significantly greater than the unexplained variance, overall. One example is the F-test in the analysis of variance.

# The Durbin Watson statistic is a test for autocorrelation in a regression model's output. 
# The statistic ranges from zero to four, with a value of 2.0 indicating zero autocorrelation.

# Jarque–Bera test is a goodness-of-fit test of whether sample data have the skewness and kurtosis matching a normal distribution.

# The adjusted r-square adjusts the value of R2 to avoid overestimating the impact of adding independent variables to the model.

There are other modules that have implemented regression analysis, such as the **sklearn** module:

As shown by the scatter plot above, it seems that the data is grouped, which are probably related to the different species present in the dataset. Let's check that.

In [ ]:
# same scatterplot but showing data per species
sns.scatterplot(x=x, y=y, hue=data["Species"])
plt.plot(x, mymodel)
plt.show()

We confirmed that the groups correspond to species. We also can see that the relationship within each group is positive, and not negative, as the overall data suggests. We can plot the regression line for each group running the following code:

In [ ]:
# Plot by group
sns.lmplot(x='Culmen Depth (mm)', y='Body Mass (g)', hue="Species", data=data)
plt.show()

Within each group, the relationship is clearly positive, despite an overall negative relationship. This paradox is called the `Simpson's Paradox` (nothing to do with the Simpson's series), as described by Simpson (1951).

It is therefore more appropriate to run the regression to each species separately.

In [ ]:
# Run same regression but only for the species 'versicolor'

x=data[(data['Species']=='Adelie Penguin (Pygoscelis adeliae)')]['Culmen Depth (mm)']
y=data[(data['Species']=='Adelie Penguin (Pygoscelis adeliae)')]['Body Mass (g)']

# Execute a method that returns some important key values of Linear Regression:
slope, intercept, r, p, std_err = stats.linregress(x, y)

# plot data with fitted line
def myfunc(x):
  return intercept + slope * x # function that returns fitted values

mymodel = list(map(myfunc, x)) # apply function to each x value

plt.scatter(x, y)
plt.plot(x, mymodel, color="red")
plt.show()

In [ ]:
# Model output table
x = sm.add_constant(x) # adding a constant (Intercept)

model = sm.OLS(y, x).fit()
predictions = model.predict(x) 

print_model = model.summary()
print(print_model)

### 2. Run multiple linear regression

When running multiple linear regression, the model will contain more than one predictor variable. The single effect of a variable may change in the presence of other variables. In the following example we will use the `Penguin` dataset to regress the body mass as a function of the remaining variables.

In [ ]:
y=data['Body Mass (g)']
x=data[["Culmen Depth (mm)", "Culmen Length (mm)", "Flipper Length (mm)"]]

x = sm.add_constant(x) # adding a constant (Intercept)

model = sm.OLS(y, x).fit()
predictions = model.predict(x) 

print_model = model.summary()
print(print_model)

We may also use the formula interface of statsmodels (like in R) to compute regression with multiple predictors. We just need append the predictors to the formula via a '+' symbol.

In [ ]:
# import formula api as alias smf 
import statsmodels.formula.api as smf

# to apply a formula the header names cannot have spaces - need to change the header names
data.rename(columns={'Body Mass (g)':'body_mass'}, inplace=True) # inplace="True" means that df will be updated
data.rename(columns={'Culmen Depth (mm)':'culmen_depth'}, inplace=True) 
data.rename(columns={'Culmen Length (mm)':'culmen_length'}, inplace=True) 
data.rename(columns={'Flipper Length (mm)':'flipper_length'}, inplace=True)

# formula: response ~ predictor1 + predictor2 + ...
model = smf.ols(formula='body_mass ~ culmen_depth + culmen_length + flipper_length', data=data).fit()
print_model = model.summary()
print(print_model)

We will now plot the partial response (or dependence) plots, which represents the effect of each variable in the presence of the remaining. This effect may be very different from the single effect, i.e., when running simple linear regression with each predictor.

In [ ]:
fig = sm.graphics.plot_partregress_grid(model)
fig.tight_layout(pad=1.0)

The chart below shows how the single effect of 'culmen depth' is very different from its joint effect with the remaining variables.

In [ ]:
# formula: response ~ predictor
model2 = smf.ols(formula='body_mass ~ culmen_depth', data=data).fit()
fig = sm.graphics.plot_partregress_grid(model2)
fig.tight_layout(pad=1.0)

The next chart uses the plot_fit function to plot the fitted values versus a chosen independent variable. It includes prediction confidence intervals and optionally plots the true dependent variable. Let's check the fit plot for each predictor.


In [ ]:
fig = sm.graphics.plot_fit(model, "culmen_depth")
fig.tight_layout(pad=1.0)

In [ ]:
fig = sm.graphics.plot_fit(model, "culmen_length")
fig.tight_layout(pad=1.0)

In [ ]:
fig = sm.graphics.plot_fit(model, "flipper_length")
fig.tight_layout(pad=1.0)

An important step when running regression is to perform a regression diagnostic to check that the assumptions are met.

In [ ]:
help(sm.graphics.influence_plot)

In [ ]:
# Diagnostic plots
# Influence plot - points with high leverage are extreme values that potentially may show a high influence on the fitted response
fig = sm.graphics.influence_plot(model, criterion="cooks") # criterion defines the size of the bubble. Cook's distance shows the influence of each observation on the fitted response values
fig.tight_layout(pad=1.0)
# point 169 has both a high leverage and cook's distance

The *statsmodel* has no specific function to run the most common diagnostic plots, but it provides a code to generate such function. Check below.

In [ ]:
# Code to produce functions to run diagnostic plots  
# https://www.statsmodels.org/dev/examples/notebooks/generated/linear_regression_diagnostics_plots.html

# base code
import numpy as np
import seaborn as sns
from statsmodels.tools.tools import maybe_unwrap_results
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from typing import Type
import statsmodels

style_talk = 'seaborn-talk'    #refer to plt.style.available

class Linear_Reg_Diagnostic():
    """
    Diagnostic plots to identify potential problems in a linear regression fit.
    Mainly,
        a. non-linearity of data
        b. Correlation of error terms
        c. non-constant variance
        d. outliers
        e. high-leverage points
        f. collinearity

    Author:
        Prajwal Kafle (p33ajkafle@gmail.com, where 3 = r)
        Does not come with any sort of warranty.
        Please test the code one your end before using.
    """


    def __init__(self,
                 results: Type[statsmodels.regression.linear_model.RegressionResultsWrapper]) -> None:
        """
        For a linear regression model, generates following diagnostic plots:

        a. residual
        b. qq
        c. scale location and
        d. leverage

        and a table

        e. vif

        Args:
            results (Type[statsmodels.regression.linear_model.RegressionResultsWrapper]):
                must be instance of statsmodels.regression.linear_model object

        Raises:
            TypeError: if instance does not belong to above object

        Example:
        >>> import numpy as np
        >>> import pandas as pd
        >>> import statsmodels.formula.api as smf
        >>> x = np.linspace(-np.pi, np.pi, 100)
        >>> y = 3*x + 8 + np.random.normal(0,1, 100)
        >>> df = pd.DataFrame({'x':x, 'y':y})
        >>> res = smf.ols(formula= "y ~ x", data=df).fit()
        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls(plot_context="seaborn-paper")

        In case you do not need all plots you can also independently make an individual plot/table
        in following ways

        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls.residual_plot()
        >>> cls.qq_plot()
        >>> cls.scale_location_plot()
        >>> cls.leverage_plot()
        >>> cls.vif_table()
        """

        if isinstance(results, statsmodels.regression.linear_model.RegressionResultsWrapper) is False:
            raise TypeError("result must be instance of statsmodels.regression.linear_model.RegressionResultsWrapper object")

        self.results = maybe_unwrap_results(results)

        self.y_true = self.results.model.endog
        self.y_predict = self.results.fittedvalues
        self.xvar = self.results.model.exog
        self.xvar_names = self.results.model.exog_names

        self.residual = np.array(self.results.resid)
        influence = self.results.get_influence()
        self.residual_norm = influence.resid_studentized_internal
        self.leverage = influence.hat_matrix_diag
        self.cooks_distance = influence.cooks_distance[0]
        self.nparams = len(self.results.params)


    def __call__(self, plot_context='seaborn-paper'):
        # print(plt.style.available)
        with plt.style.context(plot_context):
            fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
            self.residual_plot(ax=ax[0,0])
            self.qq_plot(ax=ax[0,1])
            self.scale_location_plot(ax=ax[1,0])
            self.leverage_plot(ax=ax[1,1])
            plt.show()

        self.vif_table()
        return fig, ax



    def residual_plot(self, ax=None):
        """
        Residual vs Fitted Plot

        Graphical tool to identify non-linearity.
        (Roughly) Horizontal red line is an indicator that the residual has a linear pattern
        """
        if ax is None:
            fig, ax = plt.subplots()

        sns.residplot(
            x=self.y_predict,
            y=self.residual,
            lowess=True,
            scatter_kws={'alpha': 0.5},
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        residual_abs = np.abs(self.residual)
        abs_resid = np.flip(np.sort(residual_abs))
        abs_resid_top_3 = abs_resid[:3]
        for i, _ in enumerate(abs_resid_top_3):
            ax.annotate(
                i,
                xy=(self.y_predict[i], self.residual[i]),
                color='C3')

        ax.set_title('Residuals vs Fitted', fontweight="bold")
        ax.set_xlabel('Fitted values')
        ax.set_ylabel('Residuals')
        return ax


    def qq_plot(self, ax=None):
        """
        Standarized Residual vs Theoretical Quantile plot

        Used to visually check if residuals are normally distributed.
        Points spread along the diagonal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        QQ = ProbPlot(self.residual_norm)
        QQ.qqplot(line='45', alpha=0.5, lw=1, ax=ax)

        # annotations
        abs_norm_resid = np.flip(np.argsort(np.abs(self.residual_norm)), 0)
        abs_norm_resid_top_3 = abs_norm_resid[:3]
        for r, i in enumerate(abs_norm_resid_top_3):
            ax.annotate(
                i,
                xy=(np.flip(QQ.theoretical_quantiles, 0)[r], self.residual_norm[i]),
                ha='right', color='C3')

        ax.set_title('Normal Q-Q', fontweight="bold")
        ax.set_xlabel('Theoretical Quantiles')
        ax.set_ylabel('Standardized Residuals')
        return ax

    def scale_location_plot(self, ax=None):
        """
        Sqrt(Standarized Residual) vs Fitted values plot

        Used to check homoscedasticity of the residuals.
        Horizontal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        residual_norm_abs_sqrt = np.sqrt(np.abs(self.residual_norm))

        ax.scatter(self.y_predict, residual_norm_abs_sqrt, alpha=0.5);
        sns.regplot(
            x=self.y_predict,
            y=residual_norm_abs_sqrt,
            scatter=False, ci=False,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        abs_sq_norm_resid = np.flip(np.argsort(residual_norm_abs_sqrt), 0)
        abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]
        for i in abs_sq_norm_resid_top_3:
            ax.annotate(
                i,
                xy=(self.y_predict[i], residual_norm_abs_sqrt[i]),
                color='C3')
        ax.set_title('Scale-Location', fontweight="bold")
        ax.set_xlabel('Fitted values')
        ax.set_ylabel(r'$\sqrt{|\mathrm{Standardized\ Residuals}|}$');
        return ax

    def leverage_plot(self, ax=None):
        """
        Residual vs Leverage plot

        Points falling outside Cook's distance curves are considered observation that can sway the fit
        aka are influential.
        Good to have none outside the curves.
        """
        if ax is None:
            fig, ax = plt.subplots()

        ax.scatter(
            self.leverage,
            self.residual_norm,
            alpha=0.5);

        sns.regplot(
            x=self.leverage,
            y=self.residual_norm,
            scatter=False,
            ci=False,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        leverage_top_3 = np.flip(np.argsort(self.cooks_distance), 0)[:3]
        for i in leverage_top_3:
            ax.annotate(
                i,
                xy=(self.leverage[i], self.residual_norm[i]),
                color = 'C3')

        xtemp, ytemp = self.__cooks_dist_line(0.5) # 0.5 line
        ax.plot(xtemp, ytemp, label="Cook's distance", lw=1, ls='--', color='red')
        xtemp, ytemp = self.__cooks_dist_line(1) # 1 line
        ax.plot(xtemp, ytemp, lw=1, ls='--', color='red')

        ax.set_xlim(0, max(self.leverage)+0.01)
        ax.set_title('Residuals vs Leverage', fontweight="bold")
        ax.set_xlabel('Leverage')
        ax.set_ylabel('Standardized Residuals')
        ax.legend(loc='upper right')
        return ax

    def vif_table(self):
        """
        VIF table

        VIF, the variance inflation factor, is a measure of multicollinearity.
        VIF > 5 for a variable indicates that it is highly collinear with the
        other input variables.
        """
        vif_df = pd.DataFrame()
        vif_df["Features"] = self.xvar_names
        vif_df["VIF Factor"] = [variance_inflation_factor(self.xvar, i) for i in range(self.xvar.shape[1])]

        print(vif_df
                .sort_values("VIF Factor")
                .round(2))


    def __cooks_dist_line(self, factor):
        """
        Helper function for plotting Cook's distance curves
        """
        p = self.nparams
        formula = lambda x: np.sqrt((factor * p * (1 - x)) / x)
        x = np.linspace(0.001, max(self.leverage), 50)
        y = formula(x)
        return x, y

Now we can run the diagnostic plots to our model:

In [ ]:
# Run diagnostic plots
cls = Linear_Reg_Diagnostic(model)
fig, ax = cls()

## References

Linear regression diagnostics https://www.statsmodels.org/dev/examples/notebooks/generated/linear_regression_diagnostics_plots.html

Ordinary Least Squares https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html

Regression Plots https://www.statsmodels.org/dev/examples/notebooks/generated/regression_plots.html

 Simpson, Edward H. (1951). The Interpretation of Interaction in Contingency Tables. *Journal of the Royal Statistical Society*, Series B. 13: 238–241.